# AdventureWorks EDA Postgres Queries

In [1]:
# import required libraries and set up db connection
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('postgresql://postgres@localhost:5432/adventureworks')

In [2]:
# list of schemas in db
query = '''
        SELECT schema_name
          FROM information_schema.schemata;
             
'''
schemas = pd.read_sql(query, engine)
schemas

,schema_name
0,pg_toast
1,pg_catalog
2,public
3,information_schema
4,person
5,humanresources
6,production
7,purchasing
8,sales
9,pe


In [128]:
# search for tables within schema
query = '''
        SELECT table_name, table_type
          FROM information_schema.tables
         WHERE table_schema = 'purchasing';
             
'''
tables = pd.read_sql(query, engine)
tables

,table_name,table_type
0,productvendor,BASE TABLE
1,purchaseorderdetail,BASE TABLE
2,vendor,BASE TABLE
3,purchaseorderheader,BASE TABLE
4,shipmethod,BASE TABLE
5,vvendorwithcontacts,VIEW
6,vvendorwithaddresses,VIEW


In [156]:
# search for cols within table
query = '''
        SELECT column_name
          FROM information_schema.columns
         WHERE table_name = 'vendor';
             
'''
columns = pd.read_sql(query, engine)
columns

,column_name
0,businessentityid
1,accountnumber
2,name
3,creditrating
4,preferredvendorstatus
5,activeflag
6,purchasingwebserviceurl
7,modifieddate


## Exploratory data analysis

### i. Sales, profit, qty trends

In [5]:
# sales, profit, qty_sold by year
query = '''
        SELECT EXTRACT(YEAR FROM sh.orderdate) AS year,
               ROUND(SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000000, 2) AS sales_in_millions,
               ROUND((SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) - SUM(p.standardcost * sd.orderqty)) / 1000000, 2) AS profit_in_millions,
               SUM(sd.orderqty) AS qty_sold
          FROM sales.salesorderheader sh
          JOIN sales.salesorderdetail sd ON sh.salesorderid = sd.salesorderid 
          JOIN production.product p ON sd.productid = p.productid
         GROUP BY EXTRACT(YEAR FROM sh.orderdate)
         ORDER BY year;           
'''
pd.read_sql(query, engine)

,year,sales_in_millions,profit_in_millions,qty_sold
0,2011.0,12.64,1.63,12888
1,2012.0,33.52,0.95,68579
2,2013.0,43.62,3.35,131788
3,2014.0,20.06,3.44,61659


In [192]:
# sales, profit, qty_sold by month
query = '''
        SELECT DATE_TRUNC('month', sh.orderdate) AS month,
               ROUND(SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000, 2) AS sales_in_thousands,
               ROUND((SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) - SUM(p.standardcost * sd.orderqty)) / 1000, 2) AS profit_in_thousands,
               SUM(sd.orderqty) AS qty_sold
          FROM sales.salesorderheader sh
          JOIN sales.salesorderdetail sd ON sh.salesorderid = sd.salesorderid 
          JOIN production.product p ON sd.productid = p.productid
         GROUP BY DATE_TRUNC('month', sh.orderdate)
         ORDER BY month
         LIMIT 5;         
'''
pd.read_sql(query, engine)

,month,sales_in_thousands,profit_in_thousands,qty_sold
0,2011-05-01,503.81,3.71,825
1,2011-06-01,458.91,183.78,141
2,2011-07-01,2044.60,228.95,2209
3,2011-08-01,2495.82,213.05,2904
4,2011-09-01,502.07,200.14,157


In [194]:
# MoM changes in sales & profit 

query = '''
          WITH 
          kpi_monthly AS (
              SELECT DATE_TRUNC('month', sh.orderdate) AS month,
                     ROUND(SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000, 2) AS sales_in_thousands,
                     ROUND((SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) - SUM(p.standardcost * sd.orderqty)) / 1000, 2) AS profit_in_thousands
                FROM sales.salesorderheader sh
                JOIN sales.salesorderdetail sd ON sh.salesorderid = sd.salesorderid 
                JOIN production.product p ON sd.productid = p.productid
               GROUP BY DATE_TRUNC('month', sh.orderdate)
               ORDER BY month
          )
        
          SELECT month,
                 ROUND((sales_in_thousands - LAG(sales_in_thousands) OVER(ORDER BY month)) / LAG(sales_in_thousands) OVER(ORDER BY month) * 100, 2) AS sales_mom_change,
                 ROUND((profit_in_thousands - LAG(profit_in_thousands) OVER(ORDER BY month)) / LAG(profit_in_thousands) OVER(ORDER BY month) * 100, 2) AS profit_mom_change
            FROM kpi_monthly
           LIMIT 5;
'''
pd.read_sql(query, engine)

,month,sales_mom_change,profit_mom_change
0,2011-05-01,NaN,NaN
1,2011-06-01,-8.91,4853.64
2,2011-07-01,345.53,24.58
3,2011-08-01,22.07,-6.94
4,2011-09-01,-79.88,-6.06


In [8]:
# sales, profit, qty by category
query = '''
        WITH sales_by_cat AS (
            SELECT pc.productcategoryid AS cat_id, pc.name,
                   ROW_NUMBER() OVER(ORDER BY SUM(sd.unitprice * sd.orderqty * ( 1 - sd.unitpricediscount)) DESC) AS rank,
                   SUM(sd.unitprice * sd.orderqty * ( 1 - sd.unitpricediscount)) AS sales
              FROM sales.salesorderdetail sd
              JOIN production.product p ON sd.productid = p.productid
              JOIN production.productsubcategory ps ON p.productsubcategoryid = ps.productsubcategoryid
              JOIN production.productcategory pc ON ps.productcategoryid = pc.productcategoryid
             GROUP BY pc.productcategoryid, pc.name
             ORDER BY sales DESC
          ),
             
         profit_by_cat AS (
            SELECT pc.productcategoryid AS cat_id, pc.name,
                   ROW_NUMBER() OVER(ORDER BY SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) - SUM(p.standardcost * sd.orderqty) DESC) AS rank,
                   SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) - SUM(p.standardcost * sd.orderqty)  AS profit
              FROM sales.salesorderdetail sd
              JOIN production.product p ON sd.productid = p.productid
              JOIN production.productsubcategory ps ON p.productsubcategoryid = ps.productsubcategoryid
              JOIN production.productcategory pc ON ps.productcategoryid = pc.productcategoryid
             GROUP BY pc.productcategoryid, pc.name
             ORDER BY profit DESC
          ),

         qty_by_cat AS (
             SELECT pc.productcategoryid AS cat_id, pc.name,
                    ROW_NUMBER() OVER(ORDER BY SUM(sd.orderqty) DESC) AS rank,
                    SUM(sd.orderqty) AS qty
              FROM sales.salesorderdetail sd
              JOIN production.product p ON sd.productid = p.productid
              JOIN production.productsubcategory ps ON p.productsubcategoryid = ps.productsubcategoryid
              JOIN production.productcategory pc ON ps.productcategoryid = pc.productcategoryid
             GROUP BY pc.productcategoryid, pc.name
             ORDER BY qty DESC
          )


        SELECT sbc.name AS cat_name, sbc.sales AS sales_by_cat,
               pbc.name AS cat_name, pbc.profit AS profit_by_cat, 
               qbc.name AS cat_name, qbc.qty AS qty_by_cat
          FROM sales_by_cat sbc
          JOIN profit_by_cat pbc ON sbc.rank = pbc.rank
          JOIN qty_by_cat qbc ON pbc.rank = qbc.rank;
             
    '''
pd.read_sql(query, engine)

,cat_name,sales_by_cat,cat_name,profit_by_cat,cat_name,qty_by_cat
0,Bikes,9.465117e+07,Bikes,7.936394e+06,Bikes,90268
1,Components,1.180259e+07,Accessories,6.363943e+05,Clothing,73670
2,Clothing,2.120543e+06,Components,4.902332e+05,Accessories,61932
3,Accessories,1.272073e+06,Clothing,3.088822e+05,Components,49044


In [9]:
# sales, profit, qty by subcategory
query = '''
        WITH sales_by_subcat AS (
            SELECT ps.productsubcategoryid AS subcat_id, ps.name,
                   ROW_NUMBER() OVER(ORDER BY SUM(sd.unitprice * sd.orderqty * ( 1 - sd.unitpricediscount)) DESC) AS rank,
                   SUM(sd.unitprice * sd.orderqty * ( 1 - sd.unitpricediscount)) AS sales
              FROM sales.salesorderdetail sd
              JOIN production.product p ON sd.productid = p.productid
              JOIN production.productsubcategory ps ON p.productsubcategoryid = ps.productsubcategoryid
             GROUP BY ps.productsubcategoryid, ps.name
             ORDER BY sales DESC
          ),
             
         profit_by_subcat AS (
            SELECT ps.productsubcategoryid AS subcat_id, ps.name,
                   ROW_NUMBER() OVER(ORDER BY SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) - SUM(p.standardcost * sd.orderqty) DESC) AS rank,
                   SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) - SUM(p.standardcost * sd.orderqty)  AS profit
              FROM sales.salesorderdetail sd
              JOIN production.product p ON sd.productid = p.productid
              JOIN production.productsubcategory ps ON p.productsubcategoryid = ps.productsubcategoryid
             GROUP BY ps.productsubcategoryid, ps.name
             ORDER BY profit DESC
          ),

         qty_by_subcat AS (
             SELECT ps.productsubcategoryid AS subcat_id, ps.name,
                    ROW_NUMBER() OVER(ORDER BY SUM(sd.orderqty) DESC) AS rank,
                    SUM(sd.orderqty) AS qty
              FROM sales.salesorderdetail sd
              JOIN production.product p ON sd.productid = p.productid
              JOIN production.productsubcategory ps ON p.productsubcategoryid = ps.productsubcategoryid
             GROUP BY ps.productsubcategoryid, ps.name
             ORDER BY qty DESC
          )


        SELECT sbc.name AS cat_name, sbc.sales AS sales_by_cat,
               pbc.name AS cat_name, pbc.profit AS profit_by_cat, 
               qbc.name AS cat_name, qbc.qty AS qty_by_cat
          FROM sales_by_subcat sbc
          JOIN profit_by_subcat pbc ON sbc.rank = pbc.rank
          JOIN qty_by_subcat qbc ON pbc.rank = qbc.rank
         LIMIT 5;
             
    '''
pd.read_sql(query, engine)

,cat_name,sales_by_cat,cat_name,profit_by_cat,cat_name,qty_by_cat
0,Road Bikes,4.390944e+07,Mountain Bikes,4.908042e+06,Road Bikes,47196
1,Mountain Bikes,3.644544e+07,Road Bikes,2.811074e+06,Mountain Bikes,28321
2,Touring Bikes,1.429629e+07,Mountain Frames,2.616048e+05,Jerseys,22711
3,Mountain Frames,4.713930e+06,Helmets,2.283291e+05,Helmets,19541
4,Road Frames,3.851351e+06,Touring Bikes,2.172777e+05,Tires and Tubes,18006


### ii. Customer insights

In [13]:
# total unique customer count
query = '''
        SELECT COUNT(DISTINCT customerid)
          FROM sales.salesorderheader;      
'''

pd.read_sql(query, engine)

,count
0,19119


In [127]:
# unique customer count by year (unique within year)
query = '''
        SELECT DATE_TRUNC('year', orderdate) AS year,
               COUNT(DISTINCT customerid)
          FROM sales.salesorderheader
         GROUP BY DATE_TRUNC('year', orderdate);      
'''

pd.read_sql(query, engine)

,year,count
0,2011-01-01,1406
1,2012-01-01,3162
2,2013-01-01,11095
3,2014-01-01,10354


In [11]:
# unique customer count by month (unique within month)
query = '''
        SELECT DATE_TRUNC('month', orderdate) AS month,
               COUNT(DISTINCT customerid)
          FROM sales.salesorderheader
         GROUP BY DATE_TRUNC('month', orderdate)
         LIMIT 5;      
'''

pd.read_sql(query, engine)

,date_trunc,count
0,2011-05-01,43
1,2011-06-01,141
2,2011-07-01,231
3,2011-08-01,250
4,2011-09-01,157
5,2011-10-01,327
6,2011-11-01,230
7,2011-12-01,228
8,2012-01-01,336
9,2012-02-01,219


In [195]:
# monthly running unique customers served count 
query = '''
        WITH
        first_time AS (
            SELECT customerid,
                   MIN(orderdate) AS first_purchase 
              FROM sales.salesorderheader
             GROUP BY customerid
        ),
        new_customer AS (
            SELECT DATE_TRUNC('month', sh.orderdate) AS month,
                   COUNT(DISTINCT sh.customerid)::int AS new_cust_count
              FROM sales.salesorderheader sh
              JOIN first_time ft ON ft.first_purchase = sh.orderdate
               AND ft.customerid = sh.customerid
             GROUP BY DATE_TRUNC('month', sh.orderdate)
        )

        SELECT month,
               SUM(new_cust_count) OVER(ORDER BY month) AS running_unique_customers_cnt
          FROM new_customer
         LIMIT 5;        
'''

pd.read_sql(query, engine)

,month,running_unique_customers_cnt
0,2011-05-01,43
1,2011-06-01,184
2,2011-07-01,415
3,2011-08-01,629
4,2011-09-01,786


In [104]:
# average order value
query = '''
        WITH
        order_values AS (
            SELECT sh.orderdate, sh.salesorderid,
                   SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) AS value
              FROM sales.salesorderdetail sd
              JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
             GROUP BY sh.orderdate, sh.salesorderid
        )
        SELECT ROUND(AVG(value), 2) AS avg_order_value
          FROM order_values
          
'''
pd.read_sql(query, engine)

,avg_order_value
0,3491.07


In [126]:
# average order value by year
query = '''
        WITH
        order_values AS (
            SELECT sh.orderdate, sh.salesorderid,
                   SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) AS value
              FROM sales.salesorderdetail sd
              JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
             GROUP BY sh.orderdate, sh.salesorderid
        )
        SELECT DATE_TRUNC('year', orderdate) AS year,
               ROUND(AVG(value), 2) AS avg_order_value
          FROM order_values 
         GROUP BY DATE_TRUNC('year', orderdate)
         ORDER BY DATE_TRUNC('year', orderdate)
          
'''
pd.read_sql(query, engine)

,year,avg_order_value
0,2011-01-01,7866.63
1,2012-01-01,8563.04
2,2013-01-01,3075.90
3,2014-01-01,1705.46


In [196]:
# average order value by month
query = '''
        WITH
        order_values AS (
            SELECT sh.orderdate, sh.salesorderid,
                   SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) AS value
              FROM sales.salesorderdetail sd
              JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
             GROUP BY sh.orderdate, sh.salesorderid
        )
        SELECT DATE_TRUNC('month', orderdate) AS month,
               ROUND(AVG(value), 2) AS avg_order_value
          FROM order_values 
         GROUP BY DATE_TRUNC('month', orderdate)
         ORDER BY DATE_TRUNC('month', orderdate)
         LIMIT 5;
          
'''
pd.read_sql(query, engine)

,month,avg_order_value
0,2011-05-01,11716.42
1,2011-06-01,3254.69
2,2011-07-01,8851.08
3,2011-08-01,9983.27
4,2011-09-01,3197.92


In [47]:
## total no. order 
query = '''
        SELECT COUNT(salesorderid) AS no_orders
          FROM sales.salesorderheader

'''
pd.read_sql(query, engine)

,no_orders
0,31465


In [197]:
# no. order by month
query = '''
        SELECT DATE_TRUNC('month', orderdate) AS month,
               COUNT(salesorderid) AS no_orders
          FROM sales.salesorderheader
         GROUP BY DATE_TRUNC('month', orderdate)
         ORDER BY month
         LIMIT 5;
'''
pd.read_sql(query, engine)

,month,no_orders
0,2011-05-01,43
1,2011-06-01,141
2,2011-07-01,231
3,2011-08-01,250
4,2011-09-01,157


In [200]:
# sales, profit, qty by country
query = '''
        SELECT cr.name AS country,
               ROUND(SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000, 2) AS sales_in_thousands,
               ROUND((SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) - SUM(p.standardcost * sd.orderqty)) / 1000, 2) AS profit_in_thousands,
               SUM(sd.orderqty) AS qty_sold
          FROM sales.salesorderdetail sd
          JOIN production.product p ON sd.productid = p.productid
          JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
          JOIN person.Address a ON sh.shiptoaddressid = a.addressid
          JOIN person.stateprovince sp ON a.stateprovinceid = sp.stateprovinceid
          JOIN person.countryregion cr ON sp.countryregioncode = cr.countryregioncode
         GROUP BY cr.name
         ORDER BY sales_in_thousands DESC;
'''
pd.read_sql(query, engine)

,country,sales_in_thousands,profit_in_thousands,qty_sold
0,United States,62997.59,2365.31,154092
1,Canada,16355.77,404.32,49381
2,Australia,10655.34,3434.26,18293
3,United Kingdom,7670.72,1231.91,20099
4,France,7251.56,893.09,19906
5,Germany,4915.41,1043.02,13143


In [201]:
# top 5 customers by sales
query = '''
        SELECT sh.customerid,
               CASE 
               WHEN c.storeid IS NOT NULL THEN 'Business'
               WHEN c.personid IS NOT NULL THEN 'Individual'
               ELSE 'Unknown'
               END AS customer_type,
               ROUND(SUM(sd.unitprice * sd.orderqty * (1 - sd.unitpricediscount)) / 1000, 2) AS sales_in_thousands
          FROM sales.salesorderdetail sd
          JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
          JOIN sales.customer c ON sh.customerid = c.customerid
         GROUP BY sh.customerid, c.personid, c.storeid
         ORDER BY sales_in_thousands DESC
         LIMIT 5;
'''
pd.read_sql(query, engine)

,customerid,customer_type,sales_in_thousands
0,29818,Business,877.11
1,29715,Business,853.85
2,29722,Business,841.91
3,30117,Business,816.76
4,29614,Business,799.28


In [202]:
# top 5 customers by profit
query = '''
        SELECT sh.customerid,
               CASE 
               WHEN c.storeid IS NOT NULL THEN 'Business'
               WHEN c.personid IS NOT NULL THEN 'Individual'
               ELSE 'Unknown'
               END AS customer_type,
               ROUND((SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) - SUM(p.standardcost * sd.orderqty)) / 1000, 2) AS profit_in_thousands
          FROM sales.salesorderdetail sd
          JOIN production.product p ON sd.productid = p.productid
          JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
          JOIN sales.customer c ON sh.customerid = c.customerid
         GROUP BY sh.customerid, c.personid, c.storeid
         ORDER BY profit_in_thousands DESC
         LIMIT 5;
'''
pd.read_sql(query, engine)

,customerid,customer_type,profit_in_thousands
0,30048,Business,38.70
1,30107,Business,38.67
2,29818,Business,38.04
3,29580,Business,37.92
4,29712,Business,35.85


In [203]:
# top 5 customers by qty
query = '''
        SELECT sh.customerid,
               CASE 
               WHEN c.storeid IS NOT NULL THEN 'Business'
               WHEN c.personid IS NOT NULL THEN 'Individual'
               ELSE 'Unknown'
               END AS customer_type,
               SUM(sd.orderqty) AS qty_sold
          FROM sales.salesorderdetail sd
          JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
          JOIN sales.customer c ON sh.customerid = c.customerid
         GROUP BY sh.customerid, c.personid, c.storeid
         ORDER BY qty_sold DESC
         LIMIT 5;
'''
pd.read_sql(query, engine)

,customerid,customer_type,qty_sold
0,29722,Business,2737
1,29957,Business,2554
2,30048,Business,2350
3,29712,Business,2313
4,30103,Business,2129


In [114]:
# sales by customer type
query = '''
        SELECT CASE 
               WHEN c.storeid IS NOT NULL THEN 'Business'
               WHEN c.personid IS NOT NULL THEN 'Individual'
               ELSE 'Unknown'
               END AS customer_type,
               ROUND(SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000, 2) AS sales_in_thousands,
               ROUND(SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000 / (SELECT SUM(sd.unitprice * sd.orderqty * (1 - unitpricediscount)) / 1000
                                                                                               FROM sales.salesorderdetail sd) * 100, 2) AS pct_of_sales
          FROM sales.salesorderdetail sd
          JOIN sales.salesorderheader sh ON sd.salesorderid = sh.salesorderid
          JOIN sales.customer c ON sh.customerid = c.customerid
         GROUP BY CASE 
                  WHEN c.storeid IS NOT NULL THEN 'Business'
                  WHEN c.personid IS NOT NULL THEN 'Individual'
                  ELSE 'Unknown'
                   END;
'''
pd.read_sql(query, engine)

,customer_type,sales_in_thousands,pct_of_sales
0,Business,80487.70,73.27
1,Individual,29358.68,26.73


### ii. Supplier insights

In [182]:
# purchasing total by year
query = '''
        SELECT EXTRACT(YEAR FROM ph.orderdate) AS year,
               ROUND(SUM(pd.unitprice * pd.orderqty) / 1000, 2) AS purchase_total_in_thousands
          FROM purchasing.purchaseorderdetail pd
          JOIN purchasing.purchaseorderheader ph ON pd.purchaseorderid=ph.purchaseorderid
         GROUP BY EXTRACT(YEAR FROM ph.orderdate)
         ORDER BY year;
'''
pd.read_sql(query, engine)

,year,purchase_total_in_thousands
0,2011.0,403.14
1,2012.0,3926.89
2,2013.0,20064.80
3,2014.0,39397.17


In [204]:
# purchase total, qty by month
query = '''
        SELECT DATE_TRUNC('month', ph.orderdate) AS month,
               ROUND(SUM(pd.unitprice * pd.orderqty) / 1000, 2) AS purchase_total_in_thousands,
               SUM(pd.orderqty) AS purchase_qty
          FROM purchasing.purchaseorderdetail pd
          JOIN purchasing.purchaseorderheader ph ON pd.purchaseorderid = ph.purchaseorderid
         GROUP BY DATE_TRUNC('month', ph.orderdate)
         ORDER BY month
         LIMIT 5;
'''
pd.read_sql(query, engine)

,month,purchase_total_in_thousands,purchase_qty
0,2011-04-01,103.90,3328
1,2011-12-01,299.24,12649
2,2012-01-01,700.41,24745
3,2012-02-01,328.57,9569
4,2012-03-01,646.98,24760


In [205]:
# purchase orders by month
query = '''
        SELECT DATE_TRUNC('month', ph.orderdate) AS month,
               COUNT(purchaseorderid) AS purchase_orders
          FROM purchasing.purchaseorderheader ph
         GROUP BY DATE_TRUNC('month', ph.orderdate)
         ORDER BY month
         LIMIT 5;
'''
pd.read_sql(query, engine)

,month,purchase_orders
0,2011-04-01,8
1,2011-12-01,20
2,2012-01-01,44
3,2012-02-01,24
4,2012-03-01,40


In [164]:
# purchasing total by vendor
query = '''
        SELECT v.name AS vendor,
               ROUND(SUM(pd.orderqty * pd.unitprice)/ 1000, 2) AS purchasing_total_in_thousands
          FROM purchasing.purchaseorderdetail pd
          JOIN purchasing.productvendor pv ON pd.productid = pv.productid
          JOIN purchasing.vendor v ON pv.businessentityid = v.businessentityid
         GROUP BY v.name
         ORDER BY purchasing_total_in_thousands DESC;
'''

pd.read_sql(query, engine)

,vendor,purchasing_total_in_thousands
0,Chicago City Saddles,8725.48
1,Professional Athletic Consultants,6687.74
2,"Vision Cycles, Inc.",6137.73
3,"Proseware, Inc.",6137.73
4,Sport Fan Co.,5482.95
...,...,...
81,Consumer Cycles,6.05
82,Metro Sport Equipment,5.72
83,Burnett Road Warriors,5.23
84,G & K Bicycle Corp.,4.58


In [169]:
# qty ordered by product, vendor
query = '''
        SELECT v.name AS vendor,
               pd.productid,
               SUM(pd.orderqty) AS qty_purchased
          FROM purchasing.purchaseorderdetail pd
          JOIN purchasing.productvendor pv ON pd.productid = pv.productid
          JOIN purchasing.vendor v ON pv.businessentityid = v.businessentityid
         GROUP BY v.name, pd.productid
         ORDER BY qty_purchased DESC;
'''

pd.read_sql(query, engine)

,vendor,productid,qty_purchased
0,"Proseware, Inc.",319,71500
1,West Junction Cycles,319,71500
2,"Vision Cycles, Inc.",319,71500
3,SUPERSALES INC.,325,62500
4,SUPERSALES INC.,326,62500
...,...,...,...
455,Inner City Bikes,475,42
456,Inner City Bikes,470,42
457,Bergeron Off-Roads,461,33
458,G & K Bicycle Corp.,708,25


## Export files for Tableau

In [36]:
# export sales order header table
query = '''
        SELECT *
          FROM sales.salesorderheader;
'''

orders = pd.read_sql(query, engine)
# orders.to_excel('orders.xlsx', index=False)

In [49]:
# export sales order detail table
query = '''
        SELECT *
          FROM sales.salesorderdetail;
'''
order_details = pd.read_sql(query, engine)
# order_details.to_excel('order_details.xlsx', index=False)

In [50]:
# export product table
query = '''
        SELECT *
          FROM production.product;
'''
product = pd.read_sql(query, engine)
# product.to_excel('product.xlsx', index=False)

In [53]:
# export subcategory table
query = '''
        SELECT *
          FROM production.productsubcategory;
'''
subcategory = pd.read_sql(query, engine)
# subcategory.to_excel('subcategory.xlsx', index=False)

In [55]:
# export category table
query = '''
        SELECT *
          FROM production.productcategory;
'''
category = pd.read_sql(query, engine)
# category.to_excel('category.xlsx', index=False)

In [118]:
# export customer table
query = '''
        SELECT *
          FROM sales.customer;
'''
customer = pd.read_sql(query, engine)
# customer.to_excel('customer.xlsx', index=False)

In [119]:
# export address table
query = '''
        SELECT *
          FROM person.address;
'''
address = pd.read_sql(query, engine)
# address.to_excel('address.xlsx', index=False)

In [171]:
# export stateprovince table
query = '''
        SELECT *
          FROM person.stateprovince;
'''
state_province = pd.read_sql(query, engine)
# state_province.to_excel('state_province.xlsx', index=False)

In [121]:
# export countryregion table
query = '''
        SELECT *
          FROM person.countryregion;
'''
country = pd.read_sql(query, engine)
# country.to_excel('country.xlsx', index=False)

In [172]:
# export purchaseorderdetail table
query = '''
        SELECT *
          FROM purchasing.purchaseorderdetail;
'''
purchase_detail = pd.read_sql(query, engine)
# purchasedetail.to_excel('purchase_detail.xlsx', index=False)

In [173]:
# export purchaseorderheader table
query = '''
        SELECT *
          FROM purchasing.purchaseorderheader;
'''
purchase_header = pd.read_sql(query, engine)
# purchase_header.to_excel('purchase_header.xlsx', index=False)

In [174]:
# export productvendor table
query = '''
        SELECT *
          FROM purchasing.productvendor;
'''
product_vendor = pd.read_sql(query, engine)
# product_vendor.to_excel('product_vendor.xlsx', index=False)

In [175]:
# export vendor table
query = '''
        SELECT *
          FROM purchasing.vendor;
'''
vendor = pd.read_sql(query, engine)
# vendor.to_excel('vendor.xlsx', index=False)